In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
%%writefile .env

# TAVILY_API_KEY = your_key
# JINA_API_KEY = your_key
# CHROME_INSTANCE_PATH = /Applications/Google Chrome.app/Contents/MacOS/Google Chrome
# BROWSER_HEADLESS=False

Overwriting .env


# Planning 부터 시작

In [4]:
import sys
from src.workflow import run_agent_workflow

In [9]:
# Choose workflow type: 'regular' or 'scm'
workflow_type = 'scm'  # Change to 'regular' for normal workflow

# SCM queries - Supply Chain Management examples
scm_user_query = '''
Analyze the impact of Chicago port strikes on our company's supply chain. 
Focus on lead time increases, transportation cost impacts, and potential effects on order fulfillment rates.
How should we adjust our logistics strategy and what alternative routes should we consider?
'''

scm_user_query = '''
2024년 8월 - 12월 사이시카고 항만 파업이 우리 회사 공급망에 미치는 영향을 분석해 주세요.
파업 전, 후를 비교하여 영향을 보여 주세요. 
리드타임, 운송비 주문 충족 및 이행률, 재고일수, 현금화 주기에 미칠 잠재적 영향에 집중해 주세요.
우리는 물류 전략을 어떻게 조정해야 하며, 어떤 대체 경로를 고려해야 할까요?
./data 디렉토리에 나의 회사 관련 데이터가 있어. 그중 data_descriptions.txt는 각 데이터에 대한 설명이야. 
'''
#scm_researcher_node, scm_insight_analyzer_node, scm_impact_analyzer 이건 매우매우 간단히 해줘. scm_correlation_analyzer 테스트 중이거든
# Regular analysis queries
regular_user_query = '''
AWS Cloud Market Analysis: Growth Prospects in Korea (2025-2030) 조사해줘. 결과는 pdf로 만들어줘, 
research와 coder는 정말 간단하게 1번씩만 호출 하고 조사도 아주 간단하게만 해. reporter 에이전트 테스트 중이거든
'''

# Select query based on workflow type
user_query = scm_user_query if workflow_type == 'scm' else regular_user_query

print(f"Selected workflow: {workflow_type}")
print(f"Query: {user_query[:100]}...")

# Other example queries (commented out)
'''
Other SCM example queries:
- "Analyze the impact of Suez Canal blockage on global semiconductor supply chain"
- "Assess the effects of Chinese port lockdowns on automotive industry supply chains"
- "Evaluate alternative shipping routes due to Red Sea disruptions"
- "Supply chain risk assessment for electronics manufacturing due to Taiwan tensions"

Other regular analysis queries:
- "LG 스타일러와 삼성 제품간의 비교가 필요해. 결과는 pdf로 만들어줘"
- "Transformer 알고리즘과 현 시점에서 어떤식으로 변화 발전 하는지 알려줘"
- "5-7세 아동을 위한 미술학원(홈스쿨)을 운영예정이야. 나는 미술치료와 심리학을 전공했어. 이런 전문성을 접목한 브랜딩, 마케팅 방법, 운영방안, 구체적인 프로그램(월별)을 기획해줘. 그리고 브랜드 이름도 추천해줘."
'''

Selected workflow: scm
Query: 
2024년 8월 - 12월 사이시카고 항만 파업이 우리 회사 공급망에 미치는 영향을 분석해 주세요.
파업 전, 후를 비교하여 영향을 보여 주세요. 
리드타임, 운송비 주문 충족 ...


'\nOther SCM example queries:\n- "Analyze the impact of Suez Canal blockage on global semiconductor supply chain"\n- "Assess the effects of Chinese port lockdowns on automotive industry supply chains"\n- "Evaluate alternative shipping routes due to Red Sea disruptions"\n- "Supply chain risk assessment for electronics manufacturing due to Taiwan tensions"\n\nOther regular analysis queries:\n- "LG 스타일러와 삼성 제품간의 비교가 필요해. 결과는 pdf로 만들어줘"\n- "Transformer 알고리즘과 현 시점에서 어떤식으로 변화 발전 하는지 알려줘"\n- "5-7세 아동을 위한 미술학원(홈스쿨)을 운영예정이야. 나는 미술치료와 심리학을 전공했어. 이런 전문성을 접목한 브랜딩, 마케팅 방법, 운영방안, 구체적인 프로그램(월별)을 기획해줘. 그리고 브랜드 이름도 추천해줘."\n'

In [10]:
import os
import shutil
import nest_asyncio

def remove_artifact_folder(folder_path="./artifacts/"):
    """
    ./artifact/ 폴더가 존재하면 삭제하는 함수
    
    Args:
        folder_path (str): 삭제할 폴더 경로
    """
    if os.path.exists(folder_path):
        print(f"'{folder_path}' 폴더를 삭제합니다...")
        try:
            # 폴더와 그 내용을 모두 삭제
            shutil.rmtree(folder_path)
            print(f"'{folder_path}' 폴더가 성공적으로 삭제되었습니다.")
        except Exception as e:
            print(f"오류 발생: {e}")
    else:
        print(f"'{folder_path}' 폴더가 존재하지 않습니다.")

nest_asyncio.apply()
remove_artifact_folder()

# Execute based on workflow type
if workflow_type == 'scm':
    print("🔗 Using SCM specialized workflow")
    result = run_agent_workflow(
        user_input=user_query,
        debug=False
    )
    
    # Print SCM workflow history
    print("\n=== SCM Analysis History ===")
    print("result", result)
    for history in result.get("history", []):
        print("===")
        print(f'agent: {history["agent"]}')
        print(f'message: {history["message"]}')
        
    # Show artifacts summary for SCM
    print(f"\n📁 Check ./artifacts/ folder for generated analysis files")
    if os.path.exists("./artifacts/"):
        files = [f for f in os.listdir("./artifacts/") if f.endswith('.txt')]
        print(f"Generated {len(files)} analysis files:")
        for file in sorted(files):
            print(f"  - {file}")

else:
    print("📊 Using regular analysis workflow")
    result = run_agent_workflow(
        user_input=user_query,
        debug=False
    )

    # Print the conversation history
    print("\n=== Conversation History ===")
    print("result", result)
    for history in result["history"]:
        print("===")
        print(f'agent: {history["agent"]}')
        print(f'message: {history["message"]}')


INFO [src.workflow] ===== Starting SCM workflow =====

INFO [src.workflow] 
SCM user input: 
2024년 8월 - 12월 사이시카고 항만 파업이 우리 회사 공급망에 미치는 영향을 분석해 주세요.
파업 전, 후를 비교하여 영향을 보여 주세요. 
리드타임, 운송비 주문 충족 및 이행률, 재고일수, 현금화 주기에 미칠 잠재적 영향에 집중해 주세요.
우리는 물류 전략을 어떻게 조정해야 하며, 어떤 대체 경로를 고려해야 할까요?
./data 디렉토리에 나의 회사 관련 데이터가 있어. 그중 data_descriptions.txt는 각 데이터에 대한 설명이야. 


INFO [src.graph.scm_nodes] ===== SCM Researcher starting task =====

INFO [src.utils.strands_sdk_utils] SCM_RESEARCHER - Prompt Cache Disabled


'./artifacts/' 폴더를 삭제합니다...
'./artifacts/' 폴더가 성공적으로 삭제되었습니다.
🔗 Using SCM specialized workflow
I'll help analyze the potential impacts of the Chicago port strike (August-December 2024) on supply chain operations. I'll focus on external research about the strike and its broader implications, while avoiding analysis of the company's internal data.

First, let me check if we have any existing research context: 
## Calling tool: python_repl_tool - # Cycle: 1



INFO [python_repl_tool] ===== Executing Python code =====

INFO [python_repl_tool] ===== Code execution successful =====

INFO [src.tools.decorators] Coder - Successfully executed:

```python
import os

# Check for existing research context
results_file = './artifacts/01_research_results.txt'

if os.path.exists(results_file):
    print("Found existing research file. Reading previous context...")
    try:
        with open(results_file, 'r', encoding='utf-8') as f:
            existing_content = f.read()
        
        print("=== EXISTING RESEARCH CONTEXT ===")
        print(existing_content)
        print("=== END OF EXISTING CONTEXT ===")
        
    except Exception as e:
        print(f"Error reading existing context: {e}")
else:
    print("No existing research file found. Starting fresh research.")
```


INFO [src.tools.decorators] 
Stdout: No existing research file found. Starting fresh research.



Let me start gathering information about the Chicago port strike and its potential impacts. I'll search in both English and Korean to get comprehensive coverage. 
## Calling tool: tavily_tool - # Cycle: 2



INFO [tavily_tool] ===== Searching for: Chicago port strike plans August-December 2024 impact on shipping logistics =====

INFO [tavily_tool] ===== Search successful =====

INFO [src.tools.decorators] 
Coder - Tool handle_tavily_tool returned:


# Relative Search Results

[{"title": "The 2024 US Port Strike: Lessons for Supply Chain Resilience", "url": "https://www.gep.com/blog/mind/2024-us-port-strike-lessons-for-supply-chain-resilience", "content": "A U.S. port workers strike shut down 36 East and Gulf Coast ports in October. The port congestion delayed retail, auto, and agriculture imports and is expected to impact global shipping schedules through 2025. Proactive planning strategies include warehousing, diversified port routes, and intermodal transport to mitigate future disruptions."}, {"title": "SIU logistics expert: Possible dock strikes could have staggering ...", "url": "https://news.siu.edu/2024/09/092624-siu-logistics-expert-possible-dock-strikes-could-have-staggering-impac

...

I need to update the task statuses in the existing plan based on the user's feedback indicating that the impact analysis has been completed. Let me determine which tasks I should mark as completed.

Based on the user's statement "Impact analysis completed", I need to identify which specific tasks in the plan should be marked as completed. Looking at the current plan:

The first section in the plan is:
### 1. scm_impact_analyzer: KPI 영향 정량화
- [ ] ./artifacts/01_research_results.txt와 ./artifacts/02_data_desc.txt를 검토하여 파업 영향과 가용 데이터 이해
- [ ] ./data/shipment_tracking_data.txt와 ./data/supplier_performance_data.txt를 분석하여 리드타임 영향 계산
- [ ] ./data/shipment_tracking_data.txt를 분석하여 운송비용 증가율 계산
- [ ] ./data/order_fulfillment_data.txt를 분석하여 주문 충족 및 이행률 변화 측정
- [ ] ./data/inventory_levels_data.txt를 분석하여 재고 수준 변동 및 재고일수 영향 계산
- [ ] 파업 전(2024년 8월 이전) 대비 파업 기간(8월-12월) KPI 변화 비교 분석
- [ ] 현재 추세가 지속될 경우 2025년까지의 KPI 영향 예측 시나리오 작성

Since the user mentioned "Impact analysis completed", it's reasonable to as


INFO [src.graph.scm_nodes] ===== Planner completed task =====

INFO [src.graph.scm_nodes] ===== Supervisor evaluating next action =====

INFO [src.utils.strands_sdk_utils] SUPERVISOR - Prompt Cache Disabled

INFO [src.graph.scm_nodes] message


{"next": "scm_correlation_analyzer"}


INFO [src.graph.scm_nodes] Supervisor delegating to: scm_correlation_analyzer

INFO [src.graph.scm_nodes] ===== Supervisor completed task =====

INFO [src.graph.scm_nodes] ===== SCM Correlation Analyzer starting task =====

INFO [src.utils.strands_sdk_utils] SCM_CORRELATION_ANALYZER - Prompt Cache Enabled


KeyError: 'current_time'

# SCM (Supply Chain Management) Analysis

이 섹션은 공급망 분석 전용 워크플로입니다. 

## SCM 워크플로 특징:
- **전문 에이전트**: SCM 전문가 에이전트들이 단계별 분석 수행
- **파일 기반**: 각 단계별 분석 결과를 `./artifacts/` 폴더에 저장
- **연구 출처**: 모든 연구 결과에 URL 출처 포함
- **KPI 분석**: 공급망 KPI 정량 분석 및 상관관계 분석
- **대응 전략**: 구체적인 공급망 리스크 대응 방안 제시

## 주요 분석 영역:
- 항만 파업, 운하 봉쇄 등 물류 중단 이벤트
- 운송비 증가, 리드타임 연장 등 비용 영향
- 공급업체 다변화, 대체 루트 등 대응 전략

In [ ]:
# SCM Quick Test - 간단한 SCM 분석 테스트
import os
import shutil
import nest_asyncio
from src.workflow import run_scm_workflow

# 빠른 SCM 테스트 쿼리들
scm_queries = {
    "chicago_port": "Analyze the impact of Chicago port strikes on our supply chain",
    "suez_canal": "Assess the supply chain impact of Suez Canal disruptions on electronics industry", 
    "china_lockdown": "Evaluate the effects of Chinese port lockdowns on automotive supply chains",
    "red_sea": "Analyze alternative shipping routes due to Red Sea security concerns"
}

# 테스트할 쿼리 선택
selected_query = "chicago_port"  # 원하는 쿼리 키로 변경
user_query = scm_queries[selected_query]

print(f"🔗 Testing SCM query: {selected_query}")
print(f"📝 Query: {user_query}")

# 아티팩트 폴더 정리
def remove_artifact_folder(folder_path="./artifacts/"):
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
        print(f"Cleared {folder_path}")

nest_asyncio.apply()
remove_artifact_folder()

# SCM 워크플로 실행
result = run_scm_workflow(
    user_input=user_query,
    debug=False
)

print("\n✅ SCM Analysis Complete!")
print(f"📁 Check ./artifacts/ folder for generated files")